In [1]:
%matplotlib widget

import tensorflow as tf
from matplotlib import pyplot
import matplotlib
import matplotlib.cm as colormap
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib
from sklearn import manifold
from scipy import stats

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import conv_model
import plot_batch
import generate_report_kfold
import icd_util

pyplot.style.use('dark_background')

In [2]:
group_names = icd_util.load_group_strings()
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')

In [137]:
M = metadata.loc[(slice(None), 1), 'admission_diagnosis']
plain = M[M.notna()].astype(str)
counts = plain.value_counts()

In [151]:
' '.join('alsdkfj dslf  f'.split())

'alsdkfj dslf f'

In [185]:
X = [
    (i4 if 'S/P' not in i3 and 'R/O' not in i3 else i3).replace('?', '').strip() 
    for i1 in plain.values 
    for i2 in i1.split(';') 
    for i3 in i2.split('\\')
    for i4 in i3.split('/')
]
X = [' '.join(i.split()) for i in X]
X = [i for i in X if i not in ['Sda', 'Telemetry']]
X, counts = numpy.unique(X, return_counts=True)
print(len(set(X)), 'Total')
for i in numpy.argsort(-counts)[:10]:
    print(counts[i], X[i])
counts[I[:1000]].sum() / counts.sum()

3050 Total
1007 Coronary Artery Disease
653 Congestive Heart Failure
596 Chest Pain
538 Pneumonia
473 Intracranial Hemorrhage
450 Sepsis
437 Stroke
427 Cath
426 Aortic Stenosis
382 Coronary Artery Bypass Graft


0.5852793740066022

In [143]:
# import string
# clean = lambda s: ''.join(c for c in s if c in string.ascii_lowercase)
Y = numpy.array(list(group_names.values()))
# corpus_clean = [clean(s) for s in list(X) + list(Y)]

In [201]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    analyzer='char_wb', 
    ngram_range=(2**1, 2**3)
)
Z = vectorizer.fit_transform(list(X) + list(Y))
print(Z.shape)

(12769, 108242)


In [202]:
D = Z.dot(Z.T).toarray()
DX = D[:X.size, :X.size]
DX -= numpy.diag(numpy.diag(DX))
DY = D[-Y.size:, -Y.size:]
DY -= numpy.diag(numpy.diag(DY))
DXY = D[:X.size, -Y.size:]

In [205]:
matches = numpy.argmax(DX, axis=1)
scores = numpy.max(DX, axis=1)
I = numpy.argsort(-scores)
for i in I[2500:2510]:
    print(round(scores[i], 3), '\n', X[i], '\n', X[matches[i]])

0.506 
 Right To Left Shunt 
 Right
0.506 
 Met Renal Cell Ca-Trachel Obstruction 
 Obstruction
0.505 
 Fever Tachy Cardia 
 Fever
0.505 
 Ascending Aorta To Supraciliac Aorta Bypass 
 Ascending Aorta Replacement
0.505 
 Intracutaneous Fistula 
 Enterocutaneous Fistula
0.505 
 Enterocutaneous Fistula 
 Intracutaneous Fistula
0.504 
 V-Tach 
 V-Tach, Heart Failure
0.504 
 Varicella Zoster Encephalitis 
 Varicella Pneumonia
0.504 
 Hip Fracture,Right Side 
 Right Hip Fracture
0.503 
 Icd Shocks 
 Icd Shock For Vt


In [163]:
from scipy import sparse

In [225]:
graph = sparse.csr_matrix(DX > 0.8)
count, labels = sparse.csgraph.connected_components(csgraph=graph, directed=False)
print(count, max((labels == i).sum() for i in labels))

2355 51


In [234]:
!rm clusters.txt

In [236]:
len(X)

3050

In [235]:
txt = '\n\n'.join(['\n'.join(X[labels == i]) for i in labels[1:]])
with open('clusters.txt', 'w') as f:
    f.write(txt)

In [70]:
matches = numpy.argmax(DXY, axis=1)
scores = numpy.max(DXY, axis=1)
I = numpy.argsort(-scores)
for i in I[-10:]:
    print(round(scores[i], 3), '\n', X[i], '\n', Y[matches[i]])

0.037 
 +Ett/Cath/Balloon Pump/?Cabg\Cath 
 Fetal Blood Loss
0.037 
 Exploratory Laparotomy;Lysis Of Ashesion;Bowel Resection 
 Respiratory
0.037 
 New Cva; Esrd 
 Spinal Cord
0.036 
 New Chf 
 Ulcer Of Calf
0.036 
 Cad,For Cabg\Cath 
 Mouth
0.035 
 Pvd\Aa R Unoffs 
 Ovary
0.035 
 Over Dose(Tylenol And Klonopin) 
 Reaction To Spinal Or Lumbar Puncture
0.034 
 Cads/P Cabg\Cx Angioplasty And Stent 
 Neoplasms 
0.03 
 Probable Delerium Tremons 
 Unstable Lie
0.0 
 N/V/D 
 Infectious And Parasitic Diseases 


In [49]:
mapping = {
    i: [
        (k, v) for k, v in group_names.items() 
        if i.lower() in v.lower()
    ]
    for i in counts.index
}

p1 = sum(len(v) > 0 for v in mapping.values()) / len(mapping)
p2 = sum(counts.loc[k] for k, v in mapping.items() if len(v) > 0) / counts.sum()
print(round(p1 * 100, 1))
print(round(p2 * 100, 1))

6.6
31.8


In [50]:
for i in mapping:
    if len(mapping[i]) == 0:
        mapping[i] = [
            (k, v) for k, v in group_names.items() 
            if all(j in v.lower() for j in i.lower().split())
        ]

p1 = sum(len(v) > 0 for v in mapping.values()) / len(mapping)
p2 = sum(counts.loc[k] for k, v in mapping.items() if len(v) > 0) / counts.sum()
print(round(p1 * 100, 1))
print(round(p2 * 100, 1))

8.2
33.9


In [51]:
for i in mapping:
    if len(mapping[i]) == 0:
        mapping[i] = [
            (k, v) for k, v in group_names.items() 
            if all(j in i.lower() for j in v.lower().split())
        ]

p1 = sum(len(v) > 0 for v in mapping.values()) / len(mapping)
p2 = sum(counts.loc[k] for k, v in mapping.items() if len(v) > 0) / counts.sum()
print(round(p1 * 100, 1))
print(round(p2 * 100, 1))

60.2
65.7


In [52]:
from string import ascii_lowercase
for i in mapping:
    if len(mapping[i]) == 0:
        filtered = ''.join(c for c in i.lower() if c in ascii_lowercase)
        mapping[i] = [
            (k, v) for k, v in group_names.items() 
            if all(j in filtered for j in v.lower().split())
        ]

p1 = sum(len(v) > 0 for v in mapping.values()) / len(mapping)
p2 = sum(counts.loc[k] for k, v in mapping.items() if len(v) > 0) / counts.sum()
print(round(p1 * 100, 1))
print(round(p2 * 100, 1))

61.9
66.7


In [53]:
filter_str = lambda s: ''.join(c for c in s.lower() if c in ascii_lowercase)

for i in mapping:
    if len(mapping[i]) == 0:
        mapping[i] = [
            (k, v) for k, v in group_names.items() 
            if all(j in filter_str(v) for j in i.lower().split())
        ]

p1 = sum(len(v) > 0 for v in mapping.values()) / len(mapping)
p2 = sum(counts.loc[k] for k, v in mapping.items() if len(v) > 0) / counts.sum()
print(round(p1 * 100, 1))
print(round(p2 * 100, 1))

62.1
66.8


In [ ]:
for i in mapping:
    if len(mapping[i]) == 0:
        mapping[i] = [
            (k, v) for k, v in group_names.items() 
            if all(j in filter_str(v) for j in i.lower().split())
        ]


In [25]:
p1 = sum(len(v) > 0 for v in mapping.values()) / len(mapping)
p2 = sum(counts.loc[k] for k, v in mapping.items() if len(v) > 0) / counts.sum()
print(round(p1 * 100, 1))
print(round(p2 * 100, 1))

62.1
66.8


In [119]:
mapping

{'Intracranial Hemorrhage': [('432',
   'Other And Unspecified Intracranial Hemorrhage'),
  ('432.9', 'Unspecified Intracranial Hemorrhage'),
  ('800.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
  ('800.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
  ('801.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
  ('801.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
  ('803.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
  ('803.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
  ('804.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
  ('804.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
  ('853', 'Other And Unspecified Intracranial Hemorrhage Following Injury')],
 'Pneumonia': [('003.22', 'Salmonella Pneumonia'),
  ('011.6', 'Tuberculous Pneumonia [Any Form]'),
  ('038.2', 'Pneumococcal Septicemia [Streptococcus Pneumoniae Septicemia]'),
  ('055.1', 'Postm

In [104]:
len(mapping)

3797

In [98]:
[len(v) for v in mapping.values()]

[11, 41, 1, 3, 2, 4, 0, 1, 0, 0]

In [100]:
for i in mapping:
    if len(mapping[i]) == 0:
        mapping[i] = [
            (k, v) for k, v in group_names.items() 
            if all(j in v.lower() for j in i.lower().split())
        ]

In [101]:
[len(v) for v in mapping.values()]

[11, 41, 1, 3, 2, 4, 0, 1, 0, 0]

In [90]:
[
    (k, v) for k, v in group_names.items() 
    if k[:3] == '800'
]

[('800-999', 'Injury And Poisoning '),
 ('800-804', 'Fracture Of Skull '),
 ('800-829', 'Fractures '),
 ('800', 'Fracture Of Vault Of Skull'),
 ('800.9', 'Open With Intracranial Injury Of Other And Unspecified Nature'),
 ('800.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
 ('800.7', 'Open With Subarachnoid, Subdural, And Extradural Hemorrhage'),
 ('800.6', 'Open With Cerebral Laceration And Contusion'),
 ('800.5', 'Open Without Mention Of Intracranial Injury'),
 ('800.4', 'Closed With Intracranialinjury Of Other And Unspecified Nature'),
 ('800.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
 ('800.2', 'Closed Wtih Subarachnoid, Subdural, And Extradural Hemorrhage'),
 ('800.1', 'Closed With Cerebral Laceration And Contusion'),
 ('800.0', 'Closed Without Mention Of Intracranial Injury')]

In [87]:
[
    (k, v) for k, v in group_names.items() 
    if 'intracranial' in v.lower()
    and 'hemorrhage' in v.lower()
]

[('432', 'Other And Unspecified Intracranial Hemorrhage'),
 ('432.9', 'Unspecified Intracranial Hemorrhage'),
 ('800.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
 ('800.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
 ('801.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
 ('801.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
 ('803.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
 ('803.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
 ('804.8', 'Open With Other And Unspecified Intracranial Hemorrhage'),
 ('804.3', 'Closed With Other And Unspecified Intracranial Hemorrhage'),
 ('852.5',
  'Extradural Hemorrhage Following Injury With Open Intracranial Wound'),
 ('852.4',
  'Extradural Hemorrhage Following Injury Without Mention Of Open Intracranial Wound'),
 ('852.3',
  'Subdural Hemorrhage Following Injury With Open Intracranial Wound'),
 ('852.2',
  'Subdural Hemorrhage Following Inj

In [79]:
counts[:50]

Intracranial Hemorrhage                                      428
Pneumonia                                                    419
Congestive Heart Failure                                     404
Sepsis                                                       364
Chest Pain                                                   322
Subarachnoid Hemorrhage                                      313
Coronary Artery Disease                                      311
Altered Mental Status                                        290
Stroke;Telemetry;Transient Ischemic Attack                   269
Upper Gi Bleed                                               254
Gastrointestinal Bleed                                       254
Abdominal Pain                                               215
Coronary Artery Disease\Coronary Artery Bypass Graft /Sda    184
Lower Gi Bleed                                               169
Seizure                                                      166
Pancreatitis             

In [69]:
(plain == 'nan').sum()

48040

In [62]:
counts[0] / plain.shape[0]

0.7234721845727539

In [74]:
counts[:50]

nan                                                          48040
Intracranial Hemorrhage                                        428
Pneumonia                                                      419
Congestive Heart Failure                                       404
Sepsis                                                         364
Chest Pain                                                     322
Subarachnoid Hemorrhage                                        313
Coronary Artery Disease                                        311
Altered Mental Status                                          290
Stroke;Telemetry;Transient Ischemic Attack                     269
Gastrointestinal Bleed                                         254
Upper Gi Bleed                                                 254
Abdominal Pain                                                 215
Coronary Artery Disease\Coronary Artery Bypass Graft /Sda      184
Lower Gi Bleed                                                

In [57]:
counts.shape

(3798,)

In [ ]:
len(metadata['admission_diagnosis'])

In [11]:
plain = sorted(set(metadata['admission_diagnosis'].astype(str)))
plain = [i.strip() for i in plain]

In [18]:
plain

['Aortic Stenosis\\Aortic Valve Replacement/Sda',
 'Cas',
 'Chest Pain',
 'Open Wound To The Chest/Sda',
 'Thoracic Aneurysm\\Replacement Of Ascending Aortic / Hemi Arch Under Deep Hypothermic Circulatory Arrest /Sda',
 '+ Troponin',
 '+Ett',
 '+Ett/Cath/Balloon Pump/?Cabg\\Cath',
 '+Ett;Coronary Artery Disease\\Cath',
 '+Ett;Dyspnea\\Cath',
 '+Ett\\Cath',
 '3 Vessel Coronary Artery Disease',
 '3 Vessel Disease',
 '3Rd Degree Heart Block',
 '3Rd Degree Heart Block Sarcoid',
 '4Th Ventricular Tumor/Sda',
 '90% Left Main Coronary Artery Disease',
 '? Abscess',
 '? Aortic Dissection',
 '? Bowel Perforation',
 '? Graft Infection',
 '? Psterior Fossa Mass',
 '? Small Bowel Obstruction',
 '? Volvulus',
 '?Obstruction',
 '?Sternal Issues\\Sternum Revision ? Plating/Sda',
 'A - Fib',
 'A Fib',
 'A Typical Chest Pain\\Cardiac Catheterization',
 'A-Fib,Cad,Crf',
 'A-Fib\\Tee; Cardioversion',
 'A-Flutter',
 'Aaa',
 'Aaa Cad\\Bentall Procedure;Coronary Artery Bypass Graft/Sda',
 'Aaa;',
 'Aaa;Tele

In [17]:
len(set(plain))

3788

In [12]:
len(set(plain))

3798

In [4]:
group_names

{'001-139': 'Infectious And Parasitic Diseases ',
 '001-009': 'Intestinal Infectious Diseases ',
 '001': 'Cholera',
 '001.9': 'Cholera, Unspecified',
 '001.1': 'Due To Vibrio Cholerae El Tor',
 '001.0': 'Due To Vibrio Cholerae',
 '002': 'Typhoid And Paratyphoid Fevers',
 '002.9': 'Paratyphoid Fever, Unspecified',
 '002.3': 'Paratyphoid Fever C',
 '002.2': 'Paratyphoid Fever B',
 '002.1': 'Paratyphoid Fever A',
 '002.0': 'Typhoid Fever',
 '003': 'Other Salmonella Infections',
 '003.9': 'Salmonella Infection, Unspecified',
 '003.8': 'Other Specified Salmonella Infections',
 '003.1': 'Salmonella Septicemia',
 '003.0': 'Salmonella Gastroenteritis',
 '003.2': 'Localized Salmonella Infections',
 '003.29': 'Other',
 '003.24': 'Salmonella Osteomyelitis',
 '003.23': 'Salmonella Arthritis',
 '003.22': 'Salmonella Pneumonia',
 '003.21': 'Salmonella Meningitis',
 '003.20': 'Localized Salmonella Infection, Unspecified',
 '004': 'Shigellosis',
 '004.9': 'Shigellosis, Unspecified',
 '004.8': 'Other S